In [4]:
import pymunk
import random
import pygame
import cv2
import numpy as np
import os
import pymunk.pygame_util 
import time
import json
import csv

pygame 2.2.0 (SDL 2.26.4, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
img_dir     = "/home/turgay/falling_objects_dataset/img_files/"  
csv_dir     = "/home/turgay/falling_objects_dataset/csv_files/"

In [6]:
world_width  = 512
world_height = 512
xCenter      = world_width / 2

In [7]:
def custom_color_for_shape(shape):
    # Assign random color with full opacity
    r = random.uniform(0, 1)  # Random red
    g = random.uniform(0, 1)  # Random green
    b = random.uniform(0, 1)  # Random blue
    a = 1.0                   # Full opacity
    return pymunk.SpaceDebugColor(r, g, b, a)

In [14]:
def dim():
    return random.randint(50, 100)

def xpos(prev_block):
    prev_w = prev_block[2]  # width of the previous block
    prev_x = prev_block[0]  # X position of the previous block
    return random.randint(prev_x - prev_w, prev_x + prev_w)

def ypos(prev_block, h):
    prev_y = prev_block[1]  # Y position of the previous block
    prev_h = prev_block[3]  # height of the previous block
    return prev_y - (prev_h)


def add_block(prev_block, is_first):
    w = dim()
    h = dim()
    x = xCenter if is_first else xpos(prev_block)
    y = ypos(prev_block, h)

    return (x, y, w, h)

def make_tower_world():
    
    ground_block = (xCenter, world_height - 10, world_width, 10)
    
    block1 = add_block(ground_block, True)
    block2 = add_block(block1, False)
    block3 = add_block(block2, False)
    block4 = add_block(block3, False)
    block5 = add_block(block4, False)
    return [ground_block, block1, block2, block3, block4, block5]

In [9]:
ground_block = (xCenter, world_height - 10, world_width, 10)

In [10]:
def save_img(batch, iteration, screen):
    
#---------------------------------------------------------------------------------------------
    pygame.image.save(screen, f"{img_dir}/{np.int64(iteration / 50) + (batch * 20) }.png")
#---------------------------------------------------------------------------------------------            

In [15]:
def save_csv(block_data, batch, iteration):
    
#---------------------------------------------------------------------------------------------    
    block_data2 = []
    
    blocks_for_csv       =   [data["block"] for data in block_data[:5]]
    masses_for_csv       =   [data["mass"] for data in block_data[:5]]
    colors_for_csv       =   [data["color"] for data in block_data[:5]]
        
    for block, mass, color in zip(blocks_for_csv, masses_for_csv, colors_for_csv):
        data = {
            "position_x": block[0],
            "position_y": block[1],
            "width": block[2],
            "height": block[3],
            "mass": mass, 
            "color": color,
            "time"      : iteration / 50
        }
        block_data2.append(data)

#---------------------------------------------------------------------------------------------    
    
    csv_filename = f"{csv_dir}/{np.int64(iteration / 50) + (batch * 20)}.csv"
    
    # Check if CSV file exists to append or write headers
    file_exists = os.path.exists(csv_filename)
    with open(csv_filename, 'a', newline='') as csvfile:
        fieldnames = ["position_x", "position_y", "width", "height", "mass", "color", "time"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write headers only if file doesn't exist
        if not file_exists:
            writer.writeheader()
        
        # Write the data of the blocks to the CSV file
        writer.writerows(block_data2)


In [11]:
def start_video(batch, fps=60):

    video_path   =   f"{video_dir}/{np.int64(batch) }.avi"
    
    video_writer = cv2.VideoWriter(
                                    video_path,
                                    cv2.VideoWriter_fourcc(*'XVID'),
                                    fps,
                                    (world_width, world_height)
                                    )
    return video_writer

In [12]:
def record_frames(video_writer, screen):

    frame = pygame.surfarray.array3d(screen)  # Get frame data from Pygame
    frame = np.transpose(frame, (1, 0, 2))    # Transpose to match OpenCV format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert to OpenCV color format
    video_writer.write(frame)  # Write frame to video file

In [13]:
def stop_video(video_writer):
    video_writer.release()

In [16]:
def animate_tower(batch):
    
    pygame.init()
    
    space         = pymunk.Space()
    space.gravity = (0, 9.8)

    screen        = pygame.display.set_mode((world_width, world_height))
    draw_options  = pymunk.pygame_util.DrawOptions(screen)

    # Add ground
    ground = pymunk.Segment(space.static_body, (0, world_height - 10), (world_width, world_height - 10), 1)
    space.add(ground)

    # Create blocks for the tower
    blocks     = make_tower_world()
    block_data = []

    for block in blocks:
        
        mass          = random.randint(1, 10)
        moment        = pymunk.moment_for_box(mass, (block[2], block[3]))   # mass, size(width, height)   obj= (x, y, w, h, c) 
        body          = pymunk.Body(mass, moment)
        body.position = (block[0], block[1]) 

        shape   = pymunk.Poly.create_box(body, (block[2], block[3]))
        
        r, g, b = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        shape.color = (r / 255.0, g / 255.0, b / 255.0, 1.0)

        space.add(body, shape)
        block_data.append({"mass": mass, "block": block, "color": (r, g, b)})

    
    iteration = 0
    while iteration <= 1000:  
        
        screen.fill((255, 255, 255))  
        space.debug_draw(draw_options)
        pygame.display.flip()
        
        save_img(batch, iteration, screen)
        save_csv(block_data, batch, iteration)

        space.step(1 / 60.0)  # Step the physics simulation
        iteration += 1
        
    #stop_video(video_writer) 

    pygame.quit()

In [17]:
for i in range(2):
    
    animate_tower(i) 



In [94]:
def animate_tower(batch):
    pygame.init()
    
    space         = pymunk.Space()
    space.gravity = (0, 9.8)

    screen        = pygame.display.set_mode((world_width, world_height))
    draw_options  = pymunk.pygame_util.DrawOptions(screen)

    # Add ground
    ground = pymunk.Segment(space.static_body, (0, world_height - 10), (world_width, world_height - 10), 1)
    space.add(ground)

    # Create blocks for the tower
    blocks     = make_tower_world()
    block_data = []

    for block in blocks:
        mass = random.randint(1, 10)
        moment = pymunk.moment_for_box(mass, (block[2], block[3]))  # (width, height) for box
        body = pymunk.Body(mass, moment)
        body.position = (block[0] + block[2] / 2, block[1] + block[3] / 2)  # Set to the center of the block

        shape = pymunk.Poly.create_box(body, (block[2], block[3]))
        elasticity = random.uniform(0.2, 0.8)
        shape.elasticity = elasticity

        # Assign a random color to the block
        color = random_color()

        # Draw the block with the assigned color
        shape.color = color

        space.add(body, shape)
        block_data.append({"mass": mass, "block": block, "color": color})

    iteration = 0
    while iteration <= 1000:  
        screen.fill((255, 255, 255))  # Fill the screen with white

        space.debug_draw(draw_options)  # Draw the physics simulation

        # Custom block drawing to apply colors
        for block in block_data:
            color = block["color"]
            pygame.draw.rect(screen, color, pygame.Rect(block["block"][0], block["block"][1], block["block"][2], block["block"][3]))

        pygame.display.flip()  # Update the screen
        
        # Optionally save images and data
        save_img(batch, iteration, screen)
        save_csv(block_data, batch, iteration)

        space.step(1 / 60.0)  # Step the physics simulation
        iteration += 1

    pygame.quit()

In [ ]:
for i in range(2):
    
    animate_tower(i) 



Exception ignored from cffi callback <function ext_cpSpaceDebugDrawColorForShapeImpl at 0x7155dc0fcd30>:
Traceback (most recent call last):
  File "/home/turgay/anaconda3/envs/gen_714_data/lib/python3.9/site-packages/pymunk/_callbacks.py", line 176, in ext_cpSpaceDebugDrawColorForShapeImpl
    return options.color_for_shape(shape)
  File "/home/turgay/anaconda3/envs/gen_714_data/lib/python3.9/site-packages/pymunk/space_debug_draw_options.py", line 332, in color_for_shape
    return SpaceDebugColor(*shape.color)
TypeError: <lambda>() missing 1 required positional argument: 'a'
Exception ignored from cffi callback <function ext_cpSpaceDebugDrawColorForShapeImpl at 0x7155dc0fcd30>:
Traceback (most recent call last):
  File "/home/turgay/anaconda3/envs/gen_714_data/lib/python3.9/site-packages/pymunk/_callbacks.py", line 176, in ext_cpSpaceDebugDrawColorForShapeImpl
    return options.color_for_shape(shape)
  File "/home/turgay/anaconda3/envs/gen_714_data/lib/python3.9/site-packages/pymunk/